# Purpose:

Make outputs for the Zachry Brenton Sorghum paper

# Pre-amble

In [ ]:
setwd("/scratch/")

Make a directory and get the data

In [ ]:
dir.create("sorghum/")
system("aws s3 sync s3://ddpsc-baxterlab-data/panvar/panvar_mwe/raw_inputs/sorghum/ sorghum/")

In [ ]:
system("rclone copy dandrive:WIP/panvar/sorghum/BAP_WSC/sorghum_BAP_WSC_gwas.tsv .")

# Dependencies

In [ ]:
library(devtools)

Install `panvar`

In [ ]:
load_all("~/repos/devel_panvar/")

If you need to rebuild the library for `rmd`.

The order is important here.

In [ ]:
setwd("~/repos/devel_panvar/")

In [ ]:
devtools::document()

In [ ]:
devtools::build()

In [ ]:
devtools::build_rmd("vignettes/panvaR.Rmd")

# Inputs

# Scratch

In [ ]:
gwas_table <- panvar_gwas(
    genotype_data = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz",
    phentotype_path = "sorghum/BAP_WSC_pheno.tsv"
)

In [ ]:
gwas_table %>% 
    fwrite("sorghum_BAP_WSC_gwas.tsv", sep = "\t", col.names = TRUE)

In [ ]:
gwas_table %>% 
    filter(CHROM == 4) %>%
    filter(Pvalues > 18)

In [ ]:
test_run <- panvar_func(
    phenotype_data_path = "sorghum/BAP_WSC_pheno.tsv",
    vcf_file_path = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz",
    chrom = "Chr04",
    bp = 66529675,
    all.impacts = TRUE
)